In [1]:
import numpy as np
import pandas as pd
import os
import re

from functools import reduce

In [2]:
DATA_PATH = "technology_1m_prices.csv"

In [3]:
data = pd.read_csv(DATA_PATH, index_col=0)
data.dropna(inplace=True)
data

,v,vw,o,c,h,l,t,n,ticker
0,1338056.0,58.2800,57.8425,58.6775,58.875,57.7000,2020-03-23 12:32:00,4458.0,AAPL
1,13767216.0,56.8116,58.6875,55.7975,58.875,55.5875,2020-03-23 13:02:00,24455.0,AAPL
2,41790168.0,55.7332,55.8009,55.2800,56.750,54.5000,2020-03-23 13:32:00,119429.0,AAPL
3,24935492.0,55.5366,55.2850,55.5364,56.170,55.0000,2020-03-23 14:02:00,69243.0,AAPL
4,20382612.0,55.3185,55.5625,55.0238,55.890,54.6500,2020-03-23 14:32:00,58723.0,AAPL
...,...,...,...,...,...,...,...,...,...
2734,28414.0,117.3471,117.8800,117.4500,117.880,117.2400,2022-02-18 22:30:00,308.0,TSM
2735,8285.0,117.1759,117.4000,117.4900,117.650,117.1000,2022-02-18 23:00:00,97.0,TSM
2736,2501.0,117.4259,117.3000,117.4100,117.530,117.2600,2022-02-18 23:30:00,39.0,TSM
2737,683.0,117.4410,117.5000,117.4100,117.500,117.3000,2022-02-19 00:00:00,18.0,TSM


In [4]:
data["ticker"].unique()

array(['AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA', 'NVDA', 'FB', 'TSM'],
      dtype=object)

In [5]:
close_prices = []
for ticker in data["ticker"].unique():
    close_prices.append(data[data["ticker"] == ticker][["t", "c", "ticker"]].set_index("t"))

In [6]:
current_df = close_prices[0]
for i in range(1, len(close_prices)):
    current_df = current_df.join(close_prices[i], rsuffix=f"_{close_prices[i]['ticker'].iloc[0]}", how="inner")

In [7]:
current_df = current_df.rename(columns={"c": "c_AAPL"}).drop([col for col in current_df.columns if re.search("ticker.*", col)], axis=1)

In [8]:
current_df.corr()

,c_AAPL,c_MSFT,c_GOOG,c_AMZN,c_TSLA,c_NVDA,c_FB,c_TSM
c_AAPL,1.000000,0.897821,0.827041,0.316205,0.827706,0.888518,0.421611,0.593046
c_MSFT,0.897821,1.000000,0.954454,0.478184,0.812934,0.951964,0.643755,0.527825
c_GOOG,0.827041,0.954454,1.000000,0.495090,0.718925,0.856801,0.729178,0.604520
c_AMZN,0.316205,0.478184,0.495090,1.000000,0.283836,0.431221,0.647779,0.264214
c_TSLA,0.827706,0.812934,0.718925,0.283836,1.000000,0.804004,0.284511,0.702535
c_NVDA,0.888518,0.951964,0.856801,0.431221,0.804004,1.000000,0.525818,0.411075
c_FB,0.421611,0.643755,0.729178,0.647779,0.284511,0.525818,1.000000,0.290485
c_TSM,0.593046,0.527825,0.604520,0.264214,0.702535,0.411075,0.290485,1.000000


Как видно, акции `AAPL` сильно скоррелированы с акциями `MSFT`, `GOOG`, `TSLA`, `NVDA`.